## DataClasses
Careful with dataclass usage. It can often be a sign of bad OOP. They are supposed to be used as simple data containers. They are useful because they avoid simple boilerplate code. But often they are used with complex complementing methods like in this example. Here regular classes might be more suited.  
In general, they should be used as:
 - simple datacontainers
 - scaffholding while building programs. Indeed they avoid boilerplate code and can be later transformed to regular classes when becoming more complex
 - intermediate data representation when exporting to JSON file for example

In [2]:
from dataclasses import dataclass

In [3]:
# Remember type hinting does not affect runtime
# It is used by IDLE or tools like linter to help debug while checking type

@dataclass(frozen=True) # make it a immutable dataclass 
class Stock:
    ticker : str # remember
    price : float
    dividend : float = 0
    dividend_frequency : int = 4

    @property # make it a managed attribute
    def annual_dividend(self):
        return self.dividend*self.dividend_frequency

In [4]:
from functools import total_ordering

@total_ordering
@dataclass
class Position:
    stock : Stock
    share : int

    def __eq__(self, other):
        if type(other) != Position:
            raise TypeError("Can only compare instance of Position")  
        return self.stock.price*self.share == other.stock.price*other.share


    def __gt__(self, other):
        if type(other) != Position:
            raise TypeError("Can only compare instance of Position")  
        return self.stock.price*self.share > other.stock.price*other.share

In [5]:
@dataclass
class Portfolio:
    holdings : list[Position]

    @property
    def value(self):
        return sum(position.stock.price*position.share for position in self.holdings)

    @property
    def portfolio_yield(self):
        return sum(position.stock.annual_dividend*position.share for position in self.holdings)/self.value

In [6]:
MSFT = Stock(ticker="MSFT", price=360, dividend=0.62, dividend_frequency=4)
LMT = Stock("LMT", 360, 2.80, 4)
GOOGL = Stock("GOOGL", 2200, 0, 0)

In [7]:
LMT

Stock(ticker='LMT', price=360, dividend=2.8, dividend_frequency=4)

In [8]:
from dataclasses import FrozenInstanceError

try:
    LMT.dividend = 3.2
except FrozenInstanceError as err:
    print(err)

cannot assign to field 'dividend'


In [9]:
LMT.annual_dividend

11.2

In [10]:
p1 = Position(MSFT, 100)
p2 = Position(LMT, 100)
p3 = Position(GOOGL, 10)

In [11]:
p2

Position(stock=Stock(ticker='LMT', price=360, dividend=2.8, dividend_frequency=4), share=100)

In [12]:
p2 == p1

True

In [13]:
p2 <= p3

False

In [14]:
portfolio = Portfolio(holdings=[p1, p2, p3])

In [15]:
portfolio.portfolio_yield

0.014553191489361702

In [16]:
f"{portfolio.portfolio_yield:.2%}"

'1.46%'

In [17]:
portfolio.value

94000

In [18]:
f"${portfolio.value:,.2f}"

'$94,000.00'

In [19]:
GOOGL.__dict__

{'ticker': 'GOOGL', 'price': 2200, 'dividend': 0, 'dividend_frequency': 0}